# Populate country and GDP
Pleae run kiva data population first.

This script take unique country code from Kiva data, load the country name and GDP for years after 2006 into country and country_gdp tables.

Instead of ORM, use SQL Expression Language to run queries. Also table reflection is been used. ORM is better suited to handle individual rows, and SQL Expression is better in run analytic queries and batch opertions. 


TODO: Logic: 

1) Get list of country_code from lenders
2) Get country code, name, lon/lat, and income_level from COUNTRY_URL
3) Get GDP for each year from wbdata API
 

In [100]:
import wbdata
import numpy as np
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine, Table, MetaData
from sqlalchemy.sql import select
import pymysql
import requests
import json

pymysql.install_as_MySQLdb()
from config import MYSQL_URL
engine = create_engine(MYSQL_URL, encoding='utf-8')
conn = engine.connect()

metadata = MetaData()

COUNTRY_URL = 'http://api.worldbank.org/v2/countries/{}?format=json'

In [18]:
# Table reflection
lender = Table('lender', metadata, autoload=True, autoload_with=engine)
country = Table('country', metadata, autoload=True, autoload_with=engine)
country_gdp = Table('country_gdp', metadata, autoload=True, autoload_with=engine)

In [35]:
country_code_list = pd.read_sql(select([lender.c.country_code]).distinct(), engine)['country_code'].tolist()

## Load country table

In [46]:
country_df = pd.DataFrame(columns=[
    'country_code',
    'country_name',
    'income_level',
    'longitude',
    'latitude'])

In [110]:
for country in country_code_list[1:]:
    if country is None:
        continue
    r = requests.get(COUNTRY_URL.format(country))
    try:
        data = r.json()[1][0]
        country_df = country_df.append(dict(
            country_code=country,
            income_level=data['incomeLevel']['value'],
            country_name=data['name'],
            latitude=data['latitude'] or None,
            longitude=data['longitude'] or None), 
                         ignore_index=True)
    except:
        print('Unable to find country ' + country + ', skip.')
        

error with country AI
error with country AN
error with country AQ
error with country BV
error with country CK
error with country CS
error with country CX
error with country EH
error with country FK
error with country GF
error with country GP
error with country GS
error with country IO
error with country MQ
error with country MS
error with country NF
error with country NU
error with country PN
error with country RE
error with country SH
error with country SJ
error with country TF
error with country TK
error with country UM
error with country VA
error with country WF
error with country YT


In [111]:
country_df = country_df.replace(r'^\s*$', np.nan, regex=True).drop_duplicates()

In [112]:
conn.execute('truncate country')
country_df.to_sql('country', engine, if_exists='append', index=False)

## Load country_gdp table

In [113]:
gdp_df = pd.DataFrame(columns=['country_code', 'year', 'gdp'])

In [114]:
for country_code in country_code_list:
    if country_code is None:
        continue
    try:
        for row in wbdata.get_data("NY.GDP.PCAP.CD", country=country_code):
            gdp_df = gdp_df.append({
                'country_code': country_code,
                'year': row['date'],
                'gdp': row['value']
            }, ignore_index=True)
    except Exception as e:
        print(e)

'NoneType' object is not iterable
'NoneType' object is not iterable
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
'NoneType' object is not iterable
list index out of range
list index out of range
list index out of range
'NoneType' object is not iterable


In [115]:
gdp_df.head()

,country_code,year,gdp
0,AD,2017,None
1,AD,2016,36988.6220301891
2,AD,2015,36038.2676044596
3,AD,2014,42294.9947269717
4,AD,2013,40619.7112977795


In [116]:
gdp_df = gdp_df.dropna()

In [119]:
# Only load GDP on and after 2006 as Kiva data go back to 2006
conn.execute('truncate country_gdp')
gdp_df.query('year >= "2006"').to_sql('country_gdp', engine, if_exists='append', index=False)
